In [ ]:
from src.setup.mlflow_setup import init_mlflow

In [2]:
import os
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from mlflow.models.signature import infer_signature
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# load_dotenv()

True

In [ ]:
# # -----------------------------
# # Настройка MinIO / S3 для MLflow
# # -----------------------------
# os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID", "minioadmin")
# os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY", "minioadmin")
# os.environ["MLFLOW_S3_ENDPOINT_URL"] = os.getenv("MLFLOW_S3_ENDPOINT_URL", "http://94.228.117.198:9000")

# # Подключение к MLflow Tracking Server
# MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI", "http://localhost:5000")
# mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [3]:
init_mlflow()

In [4]:
# -----------------------------
# Генерация синтетических данных
# -----------------------------
X = np.random.rand(200, 3)
y = X @ np.array([2.0, -1.5, 3.0]) + np.random.randn(200) * 0.5
feature_names = ["feat1", "feat2", "feat3"]
X = pd.DataFrame(X, columns=feature_names)
y = pd.Series(y, name="target")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -----------------------------
# Включаем автоматический логинг MLflow
# -----------------------------
mlflow.sklearn.autolog(log_input_examples=True, log_model_signatures=True)

# -----------------------------
# Эксперимент и обучение моделей
# -----------------------------
mlflow.set_experiment("auto_ml_experiment_1")

models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(n_estimators=50, random_state=42)
}

for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        # Обучение модели
        model.fit(X_train, y_train)

        # Логирование signature модели
        signature = infer_signature(X_test, model.predict(X_test))
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="model",
            signature=signature,
        )
        mse = mean_squared_error(y_test, model.predict(X_test))
        print(f"Run {model_name} finished. MSE: {mse}")

2025/10/31 17:52:25 INFO mlflow.tracking.fluent: Experiment with name 'auto_ml_experiment_1' does not exist. Creating a new experiment.
2025/10/31 17:52:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Run LinearRegression finished. MSE: 0.3226372200561786
🏃 View run LinearRegression at: http://94.228.117.198:5000/#/experiments/1/runs/ac525b3c77f84db19f1f4ce9e86f7d6b
🧪 View experiment at: http://94.228.117.198:5000/#/experiments/1


2025/10/31 17:53:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Run RandomForest finished. MSE: 0.3263401933137547
🏃 View run RandomForest at: http://94.228.117.198:5000/#/experiments/1/runs/55fc8ebbc36d406d87e20986e99a786b
🧪 View experiment at: http://94.228.117.198:5000/#/experiments/1


In [3]:
from src import init_mlflow